In [18]:
import torch
import numpy as np
from math import sin, cos

SENTENCE = "<s> I LIKE PIE VERY MUCH PIE ."
DIM_EMB = 512
torch.manual_seed(7)

vocab = dict()
i = 0
for token in SENTENCE.split():
    if token not in vocab:
        vocab[token] = i
        i += 1
vocab["<unk>"] = i
vocab["not_a_word"] = i + 1
print(vocab)

DICT_SIZE = len(vocab)

oneHotSent = np.zeros((len(SENTENCE.split()), DICT_SIZE), dtype=float)

for i in range(len(SENTENCE.split())):
    word = SENTENCE.split()[i]
    oneHotSent[i][vocab[word]] = 1

oneHotTorch = torch.as_tensor(oneHotSent, dtype=torch.float32)
    
class FUCKYOU(nn.Module):
    def __init__(self, vocab_size, batch_size):
        super().__init__()
        self.DICT_SIZE = vocab_size
        
        Wq_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
        Wq = torch.nn.Parameter(Wq_skeleton, requires_grad=True)

        Wk_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
        Wk = torch.nn.Parameter(Wk_skeleton, requires_grad=True)

        Wv_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
        Wv = torch.nn.Parameter(Wv_skeleton, requires_grad=True)
        
        Wo_skeleton = torch.tensor(np.zeros((concatTransHead.shape[1], DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        Wo = torch.nn.Parameter(Wo_skeleton, requires_grad=True)
        
        We_skeleton = torch.tensor(np.zeros((DICT_SIZE, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        We = torch.nn.Parameter(We_skeleton, requires_grad=True)
        
        Wff_skeleton = torch.tensor(np.zeros((DIM_EMB, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        Wff = torch.nn.Parameter(Wff_skeleton, requires_grad=True)
        Bff_skeleton = torch.tensor(np.zeros((DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
        Bff = torch.nn.Parameter(Bff_skeleton, requires_grad=True)
        
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
    
#DATA PREPARATION DONE

# Create self attention
# also need to add a mask here
# @TODO Add a Bias to all Wsz
def add_transformer_head_with_mask(DIM_EMB, Batch, mask):
    Wq_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wq = torch.nn.Parameter(Wq_skeleton, requires_grad=True)

    Wk_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wk = torch.nn.Parameter(Wk_skeleton, requires_grad=True)

    Wv_skeleton = torch.tensor(np.zeros((DIM_EMB, 64), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wv = torch.nn.Parameter(Wv_skeleton, requires_grad=True)

    Q = Batch.mm(Wq)
    V = Batch.mm(Wv)
    K = Batch.mm(Wk)

    tmp1 = Q.mm(torch.t(K))

    tmp = tmp1/(64**(1/2))
    #print(tmp)

    softM = torch.nn.Softmax(dim=-1)
    #apply maks here
    tmp = tmp.masked_fill(mask == 0, -1e9)

    a = softM(tmp)

    z = a.mm(V)
    return z

# @TODO add a bias to Wo
def addLinearProj(concatTransHead, DIM_EMB):
    '''Already concatenataed all heads'''
    Wo_skeleton = torch.tensor(np.zeros((concatTransHead.shape[1], DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wo = torch.nn.Parameter(Wo_skeleton, requires_grad=True)

    return concatTransHead.mm(Wo)

#Normalize
def layerNorm(mat):
    return (mat - mat.mean(dim=0, keepdim=True)) / (mat.std(dim=0, keepdim=True))

EPOCHS=10
def fwd():
    We_skeleton = torch.tensor(np.zeros((DICT_SIZE, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
    We = torch.nn.Parameter(We_skeleton, requires_grad=True)

    Batch = oneHotTorch.mm(We)
    Pos_emb = np.zeros(Batch.shape, dtype=float)
    for p in range(len(SENTENCE.split())):
        for i in range(DIM_EMB//2):
            val = p/(10000**(2*i/DIM_EMB))
            Pos_emb[p][2*i] = sin(val)
            Pos_emb[p][2*i + 1] = cos(val)

    Pos_emb_var = torch.nn.Parameter(torch.tensor(Pos_emb, dtype=torch.float32), requires_grad=False)

    Batch = Batch*(DIM_EMB**(1/2))

    Batch = Batch + Pos_emb_var

    #Batch (in this case a one sentence batch) is prepared. Maxi calls this embedded sentence

    #Create MASK HERE
    #
    mask = np.triu(np.ones((len(SENTENCE.split()), len(SENTENCE.split()))), k=1)
    mask = torch.from_numpy(mask) == 0

    transformer_head = add_transformer_head_with_mask(DIM_EMB, Batch, mask)
    postAttention = addLinearProj(transformer_head, DIM_EMB)

    # Add residual connection to the embeddings
    postAttention = postAttention + Batch

    normalizedPostAttention = layerNorm(postAttention)

    # @TODO no dropout
    # Take the output of the attention and do a one layer FF that outputs size of
    Wff_skeleton = torch.tensor(np.zeros((DIM_EMB, DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wff = torch.nn.Parameter(Wff_skeleton, requires_grad=True)
    Bff_skeleton = torch.tensor(np.zeros((DIM_EMB), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Bff = torch.nn.Parameter(Bff_skeleton, requires_grad=True)

    activation = torch.nn.ReLU()
    postFF = activation(normalizedPostAttention.mm(Wff) + Bff)

    #Residual
    preSoftmaxProj = layerNorm(postFF + normalizedPostAttention)

    #Projexct
    Wproj_skeleton = torch.tensor(np.zeros((DIM_EMB, DICT_SIZE), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Wproj = torch.nn.Parameter(Wproj_skeleton, requires_grad=True)
    Bproj_skeleton = torch.tensor(np.zeros((DICT_SIZE), dtype=float), dtype=torch.float32,).normal_(0, 1)
    Bproj = torch.nn.Parameter(Bproj_skeleton, requires_grad=True)

    linearLay = activation(preSoftmaxProj.mm(Wproj) + Bproj)

    #Softmax
    softmax = torch.nn.LogSoftmax(dim=1)
    output = softmax(linearLay)
    return output

for e in range(EPOCHS):
    truth = oneHotTorch #SAKA [:, 1:].contiguous().view(-1)
    print(truth.shape)
    #print(truth)
    print(fwd().shape)

    



{'<s>': 0, 'I': 1, 'LIKE': 2, 'PIE': 3, 'VERY': 4, 'MUCH': 5, '.': 6, '<unk>': 7, 'not_a_word': 8}
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
torch.Size([8, 9])
